# Neural Network

In [1]:
import pandas as pd
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import matplotlib.pyplot as plt
import os
import random
import warnings
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
warnings.filterwarnings('ignore')



Instructions for updating:
non-resource variables are not supported in the long term


In [2]:

# Set random seed for reproducibility
np.random.seed(42)
random.seed(42)
os.environ['PYTHONHASHSEED'] = str(42)



In [3]:
from keras import backend as K
config = tf.ConfigProto(intra_op_parallelism_threads=1,inter_op_parallelism_threads=1)
tf.set_random_seed(42)
sess = tf.Session(config=config)
K.set_session(sess)



In [4]:
import keras
from keras.layers import Dense, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler

In [5]:
data = pd.read_csv('epl_train.csv')
test = pd.read_csv('epl_test.csv')

# Preview data.
data = data[data.MW > 3]

test = test[test.MW > 3]

data.drop(['Unnamed: 0','HomeTeam', 'AwayTeam', 'Date', 'MW', 'HTFormPtsStr', 'ATFormPtsStr', 'FTHG', 'FTAG',
           'HTGS', 'ATGS', 'HTGC', 'ATGC','HomeTeamLP', 'AwayTeamLP','DiffPts','HTFormPts','ATFormPts',
           'HM4','HM5','AM4','AM5','HTLossStreak5','ATLossStreak5','HTWinStreak5','ATWinStreak5',
           'HTWinStreak3','HTLossStreak3','ATWinStreak3','ATLossStreak3'],1, inplace=True)

test.drop(['Unnamed: 0','HomeTeam', 'AwayTeam', 'Date', 'MW', 'HTFormPtsStr', 'ATFormPtsStr', 'FTHG', 'FTAG',
           'HTGS', 'ATGS', 'HTGC', 'ATGC','HomeTeamLP', 'AwayTeamLP','DiffPts','HTFormPts','ATFormPts',
           'HM4','HM5','AM4','AM5','HTLossStreak5','ATLossStreak5','HTWinStreak5','ATWinStreak5',
           'HTWinStreak3','HTLossStreak3','ATWinStreak3','ATLossStreak3'],1, inplace=True)

In [6]:

cleanup_nums = {"FTR":     {"H": 2, "D": 1, "A": 0}}

data = data.replace(cleanup_nums)

In [7]:
cleanup_nums = {"FTR":     {"H": 2, "D": 1, "A": 0}}

test = test.replace(cleanup_nums)

In [8]:
X_all = data.drop(['FTR'],1)
y_all = data['FTR']

In [9]:
XT_all = test.drop(['FTR'],1)
yT_all = test['FTR']

In [10]:
XT_all.head()

,HTP,ATP,HM1,HM2,HM3,AM1,AM2,AM3,B365H,B365D,B365A,HTGD,ATGD,DiffFormPts,DiffLP
30,1.50,1.00,W,W,L,D,L,W,7.50,5.25,1.36,0.25,-0.25,0.5,8.0
31,1.00,0.00,D,L,W,L,L,L,2.40,3.20,3.10,-0.25,-1.25,1.0,-5.0
32,2.25,0.75,W,W,W,W,L,L,1.61,4.20,5.25,2.00,0.25,1.5,-1.0
33,0.75,0.25,W,L,L,D,L,L,1.72,3.80,4.75,-0.75,-1.50,0.5,-4.0
34,1.50,0.00,L,W,W,L,L,L,1.57,4.00,6.00,0.50,-1.00,1.5,-12.0


In [11]:
#last 3 wins for both sides
X_all.HM1 = X_all.HM1.astype('str')
X_all.HM2 = X_all.HM2.astype('str')
X_all.HM3 = X_all.HM3.astype('str')
X_all.AM1 = X_all.AM1.astype('str')
X_all.AM2 = X_all.AM2.astype('str')
X_all.AM3 = X_all.AM3.astype('str')


XT_all.HM1 = XT_all.HM1.astype('str')
XT_all.HM2 = XT_all.HM2.astype('str')
XT_all.HM3 = XT_all.HM3.astype('str')
XT_all.AM1 = XT_all.AM1.astype('str')
XT_all.AM2 = XT_all.AM2.astype('str')
XT_all.AM3 = XT_all.AM3.astype('str')


#we want continous vars that are integers for our input data, so lets remove any categorical vars
def preprocess_features(X):
    ''' Preprocesses the football data and converts catagorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix = col)
                    
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print("Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns)))

Processed feature columns (27 total features):
['HTP', 'ATP', 'HM1_D', 'HM1_L', 'HM1_W', 'HM2_D', 'HM2_L', 'HM2_W', 'HM3_D', 'HM3_L', 'HM3_W', 'AM1_D', 'AM1_L', 'AM1_W', 'AM2_D', 'AM2_L', 'AM2_W', 'AM3_D', 'AM3_L', 'AM3_W', 'B365H', 'B365D', 'B365A', 'HTGD', 'ATGD', 'DiffFormPts', 'DiffLP']


In [12]:
XT_all = preprocess_features(XT_all)
print("Processed feature columns ({} total features):\n{}".format(len(XT_all.columns), list(XT_all.columns)))

Processed feature columns (27 total features):
['HTP', 'ATP', 'HM1_D', 'HM1_L', 'HM1_W', 'HM2_D', 'HM2_L', 'HM2_W', 'HM3_D', 'HM3_L', 'HM3_W', 'AM1_D', 'AM1_L', 'AM1_W', 'AM2_D', 'AM2_L', 'AM2_W', 'AM3_D', 'AM3_L', 'AM3_W', 'B365H', 'B365D', 'B365A', 'HTGD', 'ATGD', 'DiffFormPts', 'DiffLP']


In [13]:
X_all.head()


,HTP,ATP,HM1_D,HM1_L,HM1_W,HM2_D,HM2_L,HM2_W,HM3_D,HM3_L,HM3_W,AM1_D,AM1_L,AM1_W,AM2_D,AM2_L,AM2_W,AM3_D,AM3_L,AM3_W,B365H,B365D,B365A,HTGD,ATGD,DiffFormPts,DiffLP
30,1.00,1.50,1,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,1,2.38,3.25,3.20,-0.50,0.00,-0.50,-2.0
31,2.25,1.00,0,0,1,0,0,1,0,0,1,1,0,0,0,0,1,0,1,0,1.36,4.75,9.00,1.50,0.00,1.25,-5.0
32,0.75,0.75,0,1,0,0,0,1,0,1,0,0,0,1,0,1,0,0,1,0,2.10,3.30,3.60,-0.50,-1.00,0.00,-4.0
33,1.50,0.75,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,0,0,1,1.73,3.60,5.25,0.50,-0.75,0.75,-6.0
34,0.75,0.75,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,1.73,3.60,5.25,-1.25,-1.00,0.00,-8.0


In [14]:
XT_all.head()

,HTP,ATP,HM1_D,HM1_L,HM1_W,HM2_D,HM2_L,HM2_W,HM3_D,HM3_L,HM3_W,AM1_D,AM1_L,AM1_W,AM2_D,AM2_L,AM2_W,AM3_D,AM3_L,AM3_W,B365H,B365D,B365A,HTGD,ATGD,DiffFormPts,DiffLP
30,1.50,1.00,0,0,1,0,0,1,0,1,0,1,0,0,0,1,0,0,0,1,7.50,5.25,1.36,0.25,-0.25,0.5,8.0
31,1.00,0.00,1,0,0,0,1,0,0,0,1,0,1,0,0,1,0,0,1,0,2.40,3.20,3.10,-0.25,-1.25,1.0,-5.0
32,2.25,0.75,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,0,1,0,1.61,4.20,5.25,2.00,0.25,1.5,-1.0
33,0.75,0.25,0,0,1,0,1,0,0,1,0,1,0,0,0,1,0,0,1,0,1.72,3.80,4.75,-0.75,-1.50,0.5,-4.0
34,1.50,0.00,0,1,0,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1.57,4.00,6.00,0.50,-1.00,1.5,-12.0


In [15]:
from sklearn.model_selection import train_test_split



# Shuffle and split the dataset into training and testing set.
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, 
                                                    test_size = 0.2,
                                                    random_state = 2,
                                                    stratify = y_all)


# Building The Model

In [16]:
# input dimension is number of features
input_dim = X_train.shape[1]

activation_func = 'relu'
kernel_init = 'glorot_normal'
learning_rate = 0.001
batch_size = 16

model = keras.Sequential([
    Dense(48, input_shape=(input_dim,), activation=activation_func),
    Dropout(0.3),
    Dense(16),
    Dropout(0.2),
    Dense(3, activation='softmax')
])

In [17]:
es = EarlyStopping(monitor='loss', patience=3, verbose=1)

In [18]:
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam



opt = Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [19]:
model.fit(X_train, y_train, batch_size=batch_size, 
              callbacks=[es], epochs=500, verbose=0)

Epoch 15: early stopping


# Checking Accuracy

In [20]:
train_loss, train_acc = model.evaluate(X_train, y_train)
val_loss, val_acc = model.evaluate(X_test, y_test)
print('Training loss:', train_loss)
print('Training accuracy:', train_acc)
print('Validation loss:', val_loss)
print('Validation accuracy:', val_acc)

Training loss: 0.90540774229015
Training accuracy: 0.5755115
Validation loss: 0.930913714619426
Validation accuracy: 0.5675325


In [21]:
labels = ['Home', 'Draw', 'Away']
y_preds = model.predict(X_test)
y_pred_argmax = [np.argmax(i) for i in y_preds]
print(classification_report(y_test, y_pred_argmax, target_names=labels))
print(confusion_matrix(y_test, y_pred_argmax))

              precision    recall  f1-score   support

        Home       0.57      0.53      0.55       221
        Draw       0.35      0.04      0.08       189
        Away       0.58      0.86      0.69       360

    accuracy                           0.57       770
   macro avg       0.50      0.48      0.44       770
weighted avg       0.52      0.57      0.50       770

[[118   7  96]
 [ 48   8 133]
 [ 41   8 311]]


# Betting Strategy

In [22]:
funds = 100
wager = 10
favourites = 0
no_bets = 0
min_diff = 0.03

y_preds = model.predict(XT_all)

for i in range(len(XT_all)):
    addition = i + 30
    prediction = np.argmax(y_preds[i])
    print('\nPrediction', prediction)
    print('Actual', y_test.iloc[i])
    print('Favourite', np.argmin([XT_all.loc[addition,'B365H'], XT_all.loc[addition,'B365D'], XT_all.loc[addition,'B365A']]))
    print('Prediction proba', y_preds[i])
    print('Home, Draw and Away odds', XT_all.loc[addition,'B365H'], XT_all.loc[addition,'B365D'], XT_all.loc[addition,'B365A'])      
          
    
    if prediction == 0:
        odds_diff = y_preds[i][prediction] - (1/XT_all.loc[addition,'B365H'])
        if odds_diff > min_diff:
            if prediction == np.argmin([XT_all.loc[addition,'B365H'], XT_all.loc[addition,'B365D'], XT_all.loc[addition,'B365A']]): 
                                
                favourites +=1
                
            if  prediction == y_test.iloc[i]:
                funds += (wager * XT_all.loc[addition,'B365H']) - wager
            else:
                funds -= wager
        else:
            no_bets +=1
            
    elif prediction == 1:
        odds_diff = y_preds[i][prediction] - (1/XT_all.loc[addition,'B365D'])
        if odds_diff > min_diff:
            if prediction == np.argmin([XT_all.loc[addition,'B365H'], XT_all.loc[addition,'B365D'], XT_all.loc[addition,'B365A']]):  

                favourites +=1
            if  prediction == y_test.iloc[i]:
                funds +=( wager * XT_all.loc[addition,'B365D']) - wager
            else:
                funds -= wager
        else:
            no_bets +=1
    else:
        odds_diff = y_preds[i][prediction] - (1/XT_all.loc[addition,'B365A'])
        if odds_diff >  min_diff:
            if prediction == np.argmin([XT_all.loc[addition,'B365H'], XT_all.loc[addition,'B365D'],XT_all.loc[addition,'B365A']]):
                favourites +=1
            if  prediction == y_test.iloc[i]:
                funds += (wager * XT_all.loc[addition,'B365A']) - wager
            else:
                funds -= wager
        else:
            no_bets +=1
   
    print('Funds', funds)
    
print(f'Betted on favourite {favourites} times out of {len(XT_all)} matches.')
print(f'No bet placed {no_bets} times')
         


Prediction 0
Actual 2
Favourite 2
Prediction proba [0.49420965 0.24654247 0.2592479 ]
Home, Draw and Away odds 7.5 5.25 1.36
Funds 90

Prediction 2
Actual 2
Favourite 0
Prediction proba [0.2219535 0.2648214 0.5132251]
Home, Draw and Away odds 2.4 3.2 3.1
Funds 111.0

Prediction 2
Actual 2
Favourite 0
Prediction proba [0.34457117 0.14529443 0.5101344 ]
Home, Draw and Away odds 1.61 4.2 5.25
Funds 153.5

Prediction 2
Actual 0
Favourite 0
Prediction proba [0.25462463 0.25677684 0.48859844]
Home, Draw and Away odds 1.72 3.8 4.75
Funds 143.5

Prediction 2
Actual 2
Favourite 0
Prediction proba [0.06283042 0.16362299 0.77354664]
Home, Draw and Away odds 1.57 4.0 6.0
Funds 193.5

Prediction 2
Actual 0
Favourite 0
Prediction proba [0.2637089  0.21082424 0.5254668 ]
Home, Draw and Away odds 1.5 4.0 7.5
Funds 183.5

Prediction 2
Actual 1
Favourite 0
Prediction proba [0.08924849 0.44210723 0.46864432]
Home, Draw and Away odds 1.85 3.8 4.0
Funds 173.5

Prediction 0
Actual 0
Favourite 2
Prediction 

Favourite 0
Prediction proba [0.02722599 0.11717395 0.8556    ]
Home, Draw and Away odds 1.09 11.0 21.0
Funds 1251.3

Prediction 2
Actual 2
Favourite 0
Prediction proba [0.08419985 0.23224162 0.6835585 ]
Home, Draw and Away odds 2.25 3.5 3.1
Funds 1272.3

Prediction 2
Actual 0
Favourite 0
Prediction proba [0.14313518 0.34121114 0.5156536 ]
Home, Draw and Away odds 1.75 4.0 4.33
Funds 1262.3

Prediction 2
Actual 2
Favourite 0
Prediction proba [0.12281962 0.24438436 0.632796  ]
Home, Draw and Away odds 2.15 3.6 3.25
Funds 1284.8

Prediction 2
Actual 1
Favourite 2
Prediction proba [0.31497076 0.24290797 0.44212127]
Home, Draw and Away odds 2.87 3.3 2.5
Funds 1274.8

Prediction 2
Actual 0
Favourite 0
Prediction proba [0.18698186 0.28309754 0.52992064]
Home, Draw and Away odds 1.75 3.8 4.5
Funds 1264.8

Prediction 2
Actual 2
Favourite 0
Prediction proba [0.06235893 0.28430638 0.6533347 ]
Home, Draw and Away odds 2.05 3.4 3.6
Funds 1290.8

Prediction 2
Actual 1
Favourite 0
Prediction proba [

Favourite 0
Prediction proba [0.20538397 0.35028246 0.44433355]
Home, Draw and Away odds 2.5 3.5 2.7
Funds 2138.4

Prediction 2
Actual 1
Favourite 0
Prediction proba [0.34669533 0.2389468  0.41435784]
Home, Draw and Away odds 2.2 3.4 3.25
Funds 2128.4

Prediction 2
Actual 0
Favourite 0
Prediction proba [0.03061984 0.09526873 0.8741114 ]
Home, Draw and Away odds 1.33 5.25 9.0
Funds 2118.4

Prediction 0
Actual 2
Favourite 2
Prediction proba [0.38032576 0.2793837  0.34029052]
Home, Draw and Away odds 3.5 3.4 2.1
Funds 2108.4

Prediction 0
Actual 1
Favourite 2
Prediction proba [0.37431356 0.31147513 0.31421134]
Home, Draw and Away odds 2.62 3.5 2.6
Funds 2108.4

Prediction 0
Actual 0
Favourite 2
Prediction proba [0.37579706 0.27205348 0.35214943]
Home, Draw and Away odds 3.6 3.1 2.2
Funds 2134.4

Prediction 2
Actual 1
Favourite 2
Prediction proba [0.13482335 0.27095675 0.59421986]
Home, Draw and Away odds 3.0 3.3 2.4
Funds 2124.4

Prediction 0
Actual 2
Favourite 2
Prediction proba [0.44987

Prediction proba [0.26973346 0.26582882 0.46443775]
Home, Draw and Away odds 3.8 3.3 2.05
Funds 2669.1

Prediction 0
Actual 2
Favourite 2
Prediction proba [0.43821427 0.27119812 0.29058763]
Home, Draw and Away odds 4.75 4.0 1.7
Funds 2659.1

Prediction 2
Actual 2
Favourite 0
Prediction proba [0.15293424 0.34653297 0.50053287]
Home, Draw and Away odds 2.15 3.2 3.6
Funds 2685.1

Prediction 2
Actual 1
Favourite 0
Prediction proba [0.15466239 0.29561958 0.549718  ]
Home, Draw and Away odds 1.83 3.5 4.5
Funds 2675.1

Prediction 0
Actual 1
Favourite 2
Prediction proba [0.7918311  0.11686612 0.09130279]
Home, Draw and Away odds 10.0 5.0 1.33
Funds 2665.1

Prediction 2
Actual 2
Favourite 2
Prediction proba [0.29111707 0.29312772 0.4157553 ]
Home, Draw and Away odds 2.9 3.1 2.5
Funds 2665.1

Prediction 2
Actual 0
Favourite 0
Prediction proba [0.0087656  0.10964311 0.88159126]
Home, Draw and Away odds 1.5 4.2 6.5
Funds 2655.1

Prediction 2
Actual 2
Favourite 0
Prediction proba [0.18599251 0.2533

Home, Draw and Away odds 3.1 3.6 2.2
Funds 3428.1

Prediction 2
Actual 2
Favourite 0
Prediction proba [0.13503209 0.20078146 0.6641865 ]
Home, Draw and Away odds 1.4 4.75 8.0
Funds 3498.1

Prediction 2
Actual 0
Favourite 2
Prediction proba [0.34241033 0.2702508  0.38733885]
Home, Draw and Away odds 3.4 3.8 2.0
Funds 3498.1

Prediction 2
Actual 1
Favourite 2
Prediction proba [0.301223   0.26839265 0.43038437]
Home, Draw and Away odds 3.0 3.2 2.45
Funds 3498.1

Prediction 2
Actual 2
Favourite 0
Prediction proba [0.12474296 0.19849667 0.6767603 ]
Home, Draw and Away odds 1.9 3.4 4.2
Funds 3530.1

Prediction 2
Actual 2
Favourite 0
Prediction proba [0.03850866 0.08998344 0.87150794]
Home, Draw and Away odds 1.25 6.0 11.0
Funds 3630.1

Prediction 2
Actual 2
Favourite 2
Prediction proba [0.25546944 0.2666937  0.47783682]
Home, Draw and Away odds 3.3 3.3 2.2
Funds 3630.1

Prediction 0
Actual 2
Favourite 2
Prediction proba [0.5595124  0.27835697 0.16213065]
Home, Draw and Away odds 4.2 3.8 1.8
